In [ ]:
#imported libraries
import pandas as pd
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import datetime
%matplotlib inline  
from sklearn import linear_model
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#Creating of the input data
downloads = pd.read_csv('train_app_downloads.csv')
reviews = pd.read_csv('train_app_review.csv')
ratings = pd.read_csv('train_app_rating.csv')
usages = pd.read_csv('train_usage.csv')
revenues = pd.read_csv('train_revenue.csv')
output = pd.read_csv('train_final_downloads.csv')
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))

#sentiment = pd.read_csv('sentiment.csv',header=-1).ix[:,0]
#sentiment.values[sentiment.values==0.0] = 0.5
#reviews["sentiment_score"] = sentiment.values 

reviews = pd.read_csv('reviews_modified.csv').drop('Unnamed: 0', 1)


#We map -1 to 0 in the downloads (there are no 0 in the initial data)
replacementValue=0
downloads = downloads.replace(-1,replacementValue)

#Minor corrections
ratings = ratings.rename(columns={'start1': 'star1'})
ratings = pd.merge(downloads.drop(dateRange,1), ratings.drop('Unnamed: 0', 1), how='left',
                   on=["id","name","category"]).replace("NaN",replacementValue)

#This is the metric we use to determine our performance
def metric(y_pred,y_test,percent=1):
    top = int(len(y_pred)/100.0*percent)
    return (len(set([i[0] for i in sorted(enumerate(y_pred), key=lambda x:x[1],reverse=True)][0:top])
       .intersection([i[0] for i in sorted(enumerate(y_test), key=lambda x:x[1],reverse=True)][0:top])
               ))/(percent/100.0)/len(y_pred)*100
predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
predictors = predictors.fillna(0)

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
predictors['ratings_aggregate'] = 0*predictors['star1'] 

print list(predictors.columns.values) 

K = 5

cv_top_10 = []
cv_top_1  = []

for s1 in range(-5,6):
    for s2 in range(-5,6):
        for s3 in range(-5,6):
            for s4 in range(-5,6):
                for s5 in range(-5,6):
                    predictors = predictors.drop('ratings_aggregate',1)
                    predictors['ratings_aggregate'] = s1*predictors['star1'] +\
                    s2*predictors['star2'] +\
                    s3*predictors['star3'] +\
                    s4*predictors['star4'] +\
                    s5*predictors['star5'] 
                    

                    np.random.seed(1)
                    top_percent_classif = 12

                    kf = KFold(len(predictors), n_folds=K)
                    old_top = []
                    new_top = []
                    top_10 = []
                    new_top_select = [] 
                    for train, test in kf:
                        #base model
                        #old_mod=linear_model.LinearRegression(fit_intercept=False).fit(predictors.as_matrix()[train,4:12], output.as_matrix()[train,5])
                        #old_y_pred =  old_mod.predict(predictors.as_matrix()[test,4:12])
                        #old_top.append(metric(old_y_pred,output.as_matrix()[test,5]))

                        #model to determine the top 10%   (CLASSIFICATION)
                        mod_class10= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)\
                        .fit(predictors.as_matrix()[train,4:], output.as_matrix()[train,5])
                        y_pred =  mod_class10.predict(predictors.as_matrix()[test,4:])

                        #estimate of the top 10% of the test set
                        estimate_class10 = predictors.iloc[test].copy()
                        estimate_class10["firstEstimate"] = y_pred
                        estimate_class10 = estimate_class10.sort_values(by= "firstEstimate",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]
                        estimate_class10 = estimate_class10.drop("firstEstimate",1)
                        #estimate_class10 = estimate_class10.sort_values(by= "daily_avg",ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(estimate_class10))]


                        #top 10% of the trainning set
                        output_train_top_10_precent = output.iloc[train].copy().sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(1.0*top_percent_classif/100.0*len(output.iloc[train]))].drop('Unnamed: 0',1)
                        predictor_train_top_10_precent = output_train_top_10_precent.merge(predictors, how='left', on=["id","name","category","device"]).copy()
                        predictor_train_top_10_precent = predictor_train_top_10_precent.drop('cumulative_downloads_2016-02',1)
                        #predictor_train_top_10_precent = predictor_train_top_10_precent.drop('firstEstimate',1)

                        #This is the actual top 1% of the test set
                        output_test_top_1_precent = output.iloc[test].sort_values(by= 'cumulative_downloads_2016-02',ascending = False).iloc[0:int(0.01*len(output.iloc[test]))].copy()


                        #second model -> Regression on the top obtainned by regression
                        mod_top1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
                    .fit(predictor_train_top_10_precent.as_matrix()[:,4:], np.log(output_train_top_10_precent["cumulative_downloads_2016-02"].as_matrix()))

                        y_pred_2 =  mod_top1.predict(estimate_class10.as_matrix()[:,4:])

                        estimate_top1 = estimate_class10.copy()
                        estimate_top1["secondEstimate"] = y_pred_2

                        estimate_top1 = estimate_top1.sort_values(by= "secondEstimate",ascending = False).iloc[0:int(0.01*len(output.iloc[test]))]

                        estimation_error = len(estimate_top1.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent)
                        new_top.append(estimation_error)
                        top_10.append(len(estimate_class10.merge(output_test_top_1_precent, how='inner', on=["id","name","category","device"]))*100.0/len(output_test_top_1_precent))


                    cv_top_10.append(1.0*sum(top_10)/len(top_10))
                    cv_top_1.append(1.0*sum(new_top)/len(new_top))
                    print str([s1,s2,s3,s4,s5])+"  "+ str(1.0*sum(new_top)/len(new_top))

#plt.plot(range(0,11),cv_top_10,label = 'top 10 estimate')
#plt.plot(range(0,11),cv_top_1,label = 'top 1 estimate')
#plt.legend(loc = 'best')




['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'download_sum', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'korean', 'other', 'japanese', 'chinese', 'english', 'cumulative_downloads_2015-02', 'dl_projection', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'num_ratings', 'ratings_per_daily_downloads', 'num_versions', 'num_review', 'num_review_per_daily_downloads', 'gini_review', 'days_since_release', 'downloads_per_day_before', 'positive', 'negative', 'avg_sentiment_score', 'm1_max', 'm1_mean', 'm1_coef_0', 'm1_co

In [ ]:
ratings = predictors[["id","name","device","star1","star2","star3","star4","star5"]]